#Helper notebook to deploy the DLT pipeline

In [0]:
%run ../00.set_variables

In [0]:
# Databricks Notebook Script for DLT Pipeline Deployment with Relative Path

# --- 1. Install Databricks SDK (if not already available in your cluster) ---
# Ensure you are running a recent version of the SDK.
%pip install databricks-sdk

# --- 2. Import necessary libraries ---
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import pipelines
import json
import os
from pathlib import Path

# --- IMPORTANT: Access dbutils ---
# dbutils is automatically available in Databricks notebooks.
# No explicit import needed for 'dbutils' itself.

# --- 3. Configuration Parameters ---
pipeline_name = "demo_iron_ore_processing"
# This is the relative path from the *current* notebook's directory
relative_dlt_notebook_path = "../notebooks/01a. Data Engineering" # Assumes '01a. Data Engineering' is the DLT notebook

# Environment variables to set on the pipeline
environment_variables = {
    "catalog_name": catalog_name,
    "schema_name": schema_name,
    "volume_name": volume_name
}

# --- 4. Determine the full path of the target DLT notebook ---
try:
    # Get the path of the current notebook using dbutils
    # dbutils.notebook.current().path is the recommended and cleaner way
    current_notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    print(f"Current notebook path: {current_notebook_path}")

    # Use pathlib to handle path manipulation robustly
    current_notebook_dir = Path(current_notebook_path).parent
    full_dlt_notebook_path = (current_notebook_dir / relative_dlt_notebook_path).resolve()

    # Convert the Path object back to a string for the DLT API
    target_notebook_path = str(full_dlt_notebook_path)

    print(f"Resolved DLT target notebook path: {target_notebook_path}")

    # Basic validation: ensure it looks like a Databricks path
    if not target_notebook_path.startswith("/"):
        raise ValueError("Resolved DLT path does not start with '/' which is required for Databricks workspace paths.")

except Exception as e:
    print(f"Error determining target notebook path: {e}")
    raise # Re-raise to stop execution if path resolution fails

# --- 5. Initialize Databricks WorkspaceClient ---
w = WorkspaceClient()

# Check if the pipeline exists
existing_pipelines = w.pipelines.list_pipelines()

try:
    created = w.pipelines.create(
        catalog=catalog_name,
        schema=schema_name,
        continuous=False,
        name=pipeline_name,
        libraries=[pipelines.PipelineLibrary(notebook=pipelines.NotebookLibrary(path=target_notebook_path))],
        serverless=True,
        configuration=environment_variables
    )
    print("creating a new pipeline")
    pipeline_id = created.pipeline_id
#crude fallback to update to save time on initial creation
except Exception as e:
    print(f"pipeline exists... updating existing got this error trying to create : {e}")
    pipeline_id = next(pipeline.id for pipeline in existing_pipelines if pipeline.name == pipeline_name)
    updated = w.pipelines.update(
        pipeline_id=pipeline_id,
        catalog=catalog_name,
        schema=schema_name,
        continuous=False,
        name=pipeline_name,
        libraries=[pipelines.PipelineLibrary(notebook=pipelines.NotebookLibrary(path=target_notebook_path))],
        serverless=True,
        configuration=environment_variables
    )
    pipeline_id = created.pipeline_id
    
#run the new or updated pipeline
run = w.pipelines.start_update(pipeline_id=pipeline_id)
display(run)
    